# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import time
from vaastav.fbref import *

# Getting raw data

In [3]:
def get_url( url ):
    df = pd.read_html( url, header=1)[0]
    return df

In [4]:
def get_shorter_season(season):
    res = season.split("-")
    return res[0] + "-" + res[1][2:]

In [5]:
get_shorter_season("2022-2023")

'2022-23'

In [6]:
def get_logs_for_players_in_season(players, season, players_names=[]):
    logs = pd.DataFrame({'A' : []})
    for id, player in list(players.items()):
        player_name = player.data[0]['player'].replace(" ", "-").replace("ï", "i").replace("é", "e").replace("á", "a").replace("ó", "o").replace("Á", "A").replace("ø", "o").replace("ć", "c").replace("í", "i").replace("ú", "u").replace("Ć", "C").replace("ã", "a").replace("ğ", "g").replace("ş", "s").replace("É", "E").replace("ñ", "n").replace("Ł", "L").replace("ń", "n").replace("ß", "ss").replace("ç", "c").replace("İ", "I").replace("č", "c").replace("ö", "o").replace("ë", "e").replace("š", "s").replace("ä", "a").replace("Ç", "C").replace("ü", "u").replace("Ø", "O").replace("ú", "u").replace("ú", "u").replace("î", "i")
        if players_names and player_name not in players_names: 
            continue
        print("Getting data for " + player_name)
        url = 'https://fbref.com/en/players/' + str(id) + '/matchlogs/' + season + '/summary/' + player_name + '-Match-Logs'
        new_player_df = get_url(url)
        new_player_df["Name"] = player_name
        new_player_df["Season"] = get_shorter_season(season)
        
        
        # dropping NaN rows
        new_player_df = new_player_df[new_player_df['Date'].notna()]
        
        # only Premier League
        new_player_df = new_player_df[new_player_df.Comp == "Premier League"]
        
        # only last 15 matches
        new_player_df = new_player_df[-15:]
        
        if not logs.empty:
            logs = pd.concat([logs, new_player_df])
        else:
            logs = new_player_df
        
        # to not get timeouted
        time.sleep(3.2)
    return logs

In [7]:
players, stats = get_epl_players()

In [ ]:
%%time
# player_names = ["Mohamed-Salah", "Miguel-Almiron", "Bukayo-Saka", "Kevin-De-Bruyne", "Wilfried-Zaha", "Mason-Mount", "Bruno-Fernandes"]
logs = get_logs_for_players_in_season(players, '2022-2023', None)

Getting data for Brenden-Aaronson
Getting data for Che-Adams
Getting data for Tyler-Adams
Getting data for Tosin-Adarabioyo
Getting data for Nayef-Aguerd
Getting data for Rayan-Ait-Nouri
Getting data for Kristoffer-Ajer
Getting data for Manuel-Akanji
Getting data for Nathan-Ake
Getting data for Marc-Albrighton
Getting data for Thiago-Alcantara
Getting data for Carlos-Alcaraz
Getting data for Trent-Alexander-Arnold
Getting data for Alisson
Getting data for Dele-Alli
Getting data for Miguel-Almiron
Getting data for Julian-Alvarez
Getting data for Daniel-Amartey
Getting data for Joachim-Andersen
Getting data for Elliot-Anderson
Getting data for Jaidon-Anthony
Getting data for Michail-Antonio
Getting data for Antony
Getting data for Cameron-Archer
Getting data for Alphonse-Areola
Getting data for Joe-Aribo
Getting data for Adam-Armstrong
Getting data for Stuart-Armstrong
Getting data for Kepa-Arrizabalaga
Getting data for Pierre-Emerick-Aubameyang
Getting data for Ludwig-Augustinsson
Getti

# Modyfing data

In [ ]:
# splitting result column
logs[['WDL', 'GoalsTeams']] = logs['Result'].str.split(' ', expand=True)
logs[['Team Score', 'Opp Score']] = logs['GoalsTeams'].str.split('–', expand=True)
logs['Team Score'] = logs['Team Score'].astype(int)
logs['Opp Score'] = logs['Opp Score'].astype(int)
logs = logs.drop(['Result'], axis=1)

In [ ]:
logs.replace("On matchday squad, but did not play", 0, inplace=True)
logs["Min"] = logs["Min"].astype(float)
logs["Gls"] = logs["Gls"].astype(float)
logs["Ast"] = logs["Ast"].astype(float)

In [ ]:
# additional columns
# approximated FPL points
logs["CS"] = (logs["Opp Score"] == 0).astype(float)
logs["Was Home"] = (logs["Venue"] == "Home").astype(float)
logs["GW"] = logs['Round'].str.split(' ', expand=True)[1]
logs["FPL"] = 1 + (logs["Min"] >= 60) + (5 * logs["Gls"]) + (3 * logs["Ast"]) + (1 * logs["CS"])

In [ ]:
logs

# Saving to csv

In [ ]:
logs.to_csv("data/logs_all.csv")

# Features

In [ ]:
info = ["Date", "Day", "GW", "Was Home"]
features = ["Min", "Gls", "Sh", "SoT", "xG", "npxG", "xAG", "CS"]
to_predict = ["FPL"]

In [ ]:
logs[info + features + to_predict]

In [ ]:
logs[info + features + to_predict].to_csv("data/logs_features.csv")